In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import methods.selenium_patch as selenium_patch
from datetime import timedelta
from methods.scraper import Ticker

Patching stockdex selenium interface for aarch64 Linux compatibility...
Patch applied: Selenium configured and Parsing logic updated.


In [6]:
ticker = "RHM.DE"
data = Ticker(ticker).key_financial_ratios(frequency="quarterly")
display(data)

,"(Fiscal Quarter, Period Ending)",Market Capitalization,Market Cap Growth,Enterprise Value,Last Close Price,PE Ratio,Forward PE,PS Ratio,PB Ratio,P/TBV Ratio,P/FCF Ratio,...,Return on Equity (ROE),Return on Assets (ROA),Return on Invested Capital (ROIC),Return on Capital Employed (ROCE),Earnings Yield,FCF Yield,Dividend Yield,Payout Ratio,Buyback Yield / Dilution,Total Shareholder Return
Current,"Feb '26 Feb 11, 2026",73043,126.61%,75396,1592.00,84.52,43.48,6.64,14.18,38.32,745.33,...,16.05%,5.59%,3.49%,20.30%,1.15%,0.13%,0.50%,43.93%,0.05%,0.55%
Q3 2025,"Sep '25 Sep 30, 2025",91051,331.15%,93167,1984.50,108.52,54.88,8.28,17.67,47.77,-,...,13.57%,4.15%,3.04%,20.50%,0.92%,0.11%,-,-,-3.75%,-3.75%
Q2 2025,"Jun '25 Jun 30, 2025",81832,296.21%,82915,1797.00,99.55,55.22,7.67,16.65,47.11,-,...,9.20%,3.08%,2.35%,20.30%,1.00%,0.47%,-,281.68%,-6.77%,-6.77%
Q1 2025,"Mar '25 Mar 31, 2025",57294,153.28%,58878,1312.36,76.09,44.68,5.47,11.62,33.92,39.08,...,50.41%,14.39%,10.83%,19.70%,1.31%,2.56%,-,-,-7.49%,-7.49%
Q4 2024,"Dec '24 Dec 31, 2024",26684,114.32%,28393,611.51,37.22,21.87,2.74,5.98,16.33,27.01,...,18.87%,5.83%,4.02%,18.70%,2.69%,3.70%,1.32%,-,-,1.32%
Q3 2024,"Sep '24 Sep 30, 2024",21118,99.59%,22899,483.95,86.20,18.80,3.37,5.67,16.05,274.26,...,16.57%,4.99%,3.49%,10.00%,1.16%,0.36%,-,-,-2.41%,-2.41%
Q2 2024,"Jun '24 Jun 30, 2024",20654,89.83%,22258,473.30,97.42,20.36,3.71,5.72,17.15,-143.43,...,6.01%,2.49%,1.55%,8.50%,1.03%,-0.70%,-,400.00%,-0.17%,-0.17%
Q1 2024,"Mar '24 Mar 31, 2024",22621,90.58%,23946,512.71,108.75,24.48,4.68,6.15,17.63,-,...,-,-,-,6.50%,0.92%,-2.41%,-,-,-0.09%,-0.09%
Q4 2023,"Dec '23 Dec 31, 2023",12451,54.92%,14672,282.43,-,16.16,-,3.42,8.12,-,...,14.08%,4.12%,2.87%,-,-,-,-,-,-,-
Q3 2023,"Sep '23 Sep 30, 2023",10581,54.53%,11611,240.02,48.99,14.73,2.29,3.28,11.11,-,...,9.37%,3.12%,2.27%,6.50%,2.04%,-4.35%,-,-,1.21%,1.21%


In [7]:
prices = yf.Ticker(ticker).history("max").get("Close")
earning_prices = []
for date in data.index:
    got_price = False
    i = 0
    while got_price == False and i<7:
        try:
            earning_prices.append(prices[pd.to_datetime(date).tz_localize("America/New_York") - timedelta(days=i)]) # type: ignore
            got_price = True
        except:
            i += 1
    if got_price == False:
        earning_prices.append(np.nan)

earning_changes = [np.nan]
for i in range(1, len(earning_prices)):
    earning_changes.append((earning_prices[i-1]/earning_prices[i]-1)*100)

In [8]:
data["Future Change%"] = earning_changes
data["Price"] = earning_prices
data["Ticker"] = ticker
data = data[data.columns.to_list()[::-1]]
data

field_name,Ticker,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,Book Value Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,...,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,EBITDA Margin,EBIT Margin,Operating Margin,Gross Margin,Debt/Equity Ratio,Long-term Debt / Capital,Current Ratio
2025-09-30,DNKEY,21.440001,NaN,-3.4607,-3.4418,16.76100,0.38210,0.15000,3.25550,3.11690,...,0.00370,40.30670,53.91750,,,,,8.86690,0.87740,
2025-06-30,DNKEY,20.389999,5.149589,-5.15,-5.1483,15.80990,0.35790,0.14580,3.29530,3.15680,...,0.00370,38.99610,52.83860,,,,,9.76080,0.88660,
2025-03-31,DNKEY,16.500000,23.575754,4.43110,4.43710,14.31540,0.37410,0.15320,3.54400,3.39790,...,0.00370,41.32510,54.49000,,,,,10.26030,0.88990,
2024-12-31,DNKEY,13.301474,24.046406,-7.208,-7.1483,15.27590,0.39070,0.15880,3.49210,3.35820,...,0.00380,41.70050,55.57200,,,,,9.63030,0.88370,
2024-09-30,DNKEY,13.677350,-2.748167,-0.2609,-0.2552,15.50580,0.41160,0.16470,3.61050,3.48470,...,0.00370,44.66740,57.49170,,,,,9.54660,0.88190,
2024-06-30,DNKEY,13.216092,3.490123,-0.2438,-0.2336,15.13520,0.38790,0.15510,3.39470,3.27880,...,0.00370,41.52920,55.29160,,,,,9.54450,0.88170,
2024-03-31,DNKEY,13.418335,-1.507213,-0.26130,-0.25250,14.74390,0.38990,0.15170,3.36320,3.24590,...,0.00380,40.39470,53.94330,,,,,9.62170,0.87990,
2023-12-31,DNKEY,11.291124,18.839670,-3.2682,-3.2587,14.83950,0.40300,0.15230,3.38520,3.26840,...,0.00390,39.25050,49.30220,,,,,9.69660,0.87670,
2023-09-30,DNKEY,9.875496,14.334758,6.9528,6.9759,14.33410,0.37540,0.13890,3.27180,3.15400,...,0.00360,38.67240,47.07720,,,,,10.54080,0.88100,
2023-06-30,DNKEY,9.932787,-0.576787,-0.3844,-0.3736,14.34570,0.35430,0.13430,3.08270,2.97130,...,0.00370,36.62480,43.98480,,,,,10.23230,0.88080,


In [12]:
def imputer(df: pd.DataFrame, max_nans_share: float) -> pd.DataFrame:
    symbol = df.loc[df.index[0], "Ticker"]
    df = df.drop("Ticker", axis=1)
    
    # go through all data points and count nans
    row_nans = [0 for _ in df.index]
    col_nans = [0 for _ in df.columns]
    i = 0
    for row in df.index:
        j = 0
        for column in df.columns:
            val = df.loc[row, column]
            if val == '':
                val = np.nan
                df.loc[row, column] = np.nan
            if np.isnan(float(val)): # type: ignore
                row_nans[i] += 1
                col_nans[j] += 1
            else:
                df.loc[row, column] = float(df.loc[row, column]) # type: ignore
            j += 1
        i += 1
                
    # delete rows and columns with too many nans or missing price
    drop_rows = []
    for i in range(len(df.index)):
        if row_nans[i]/len(df.columns) > max_nans_share or np.isnan(df.loc[df.index[i], "Price"]): # type: ignore
            drop_rows.append(df.index[i])
    drop_cols = []
    for j in range(len(df.columns)):
        if col_nans[j]/len(df.index) > max_nans_share:
            drop_cols.append(df.columns[j])
    df = df.drop(drop_rows)
    df = df.drop(drop_cols, axis=1)

    # impute last nan values
    for j in range(len(df.columns)):
        if df.columns[j] in ["Future Change%"]:
            continue
        impute_indices = []
        for i in range(len(df.index)):
            if np.isnan(df.iloc[i, j]): # type: ignore
                impute_indices = [i]
                u = i-1 if i != 0 else 0
                l = i+1 if i != len(df.index)-1 else len(df.index)-1
                while np.isnan(df.iloc[i, j]): # type: ignore
                    if np.isnan(df.iloc[u, j]) == False and np.isnan(df.iloc[l, j]) == False: # type: ignore
                        for k in impute_indices:
                            df.iloc[k, j] = (df.iloc[u, j]+df.iloc[l, j])/2 # type: ignore
                    if np.isnan(df.iloc[u, j]) == False and l == len(df.index)-1: # type: ignore
                        for k in impute_indices:
                            df.iloc[k, j] = df.iloc[u, j] # type: ignore
                    if u == 0 and np.isnan(df.iloc[l, j]) == False: # type: ignore
                        for k in impute_indices:
                            df.iloc[k, j] = df.iloc[l, j] # type: ignore
                    if np.isnan(df.iloc[u, j]): # type: ignore
                        impute_indices.append(u) # type: ignore
                        if u != 0:
                            u += -1 
                    if np.isnan(df.iloc[l, j]): # type: ignore
                        impute_indices.append(l) # type: ignore
                        if l != len(df.index)-1:
                            l += 1

    df.insert(0, "Ticker", symbol)
    return df

imputer(data, max_nans_share=0.3)

field_name,Ticker,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,Book Value Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,Debt/Equity Ratio,Long-term Debt / Capital


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import numpy as np
import copy

# 1. Prepare Data
# Re-run imputer to ensure we have the dataframe
df_model = imputer(data, max_nans_share=0.3)
target_col = "Future Change%"

# Separate Features and Target
X_cols = [c for c in df_model.columns if c != target_col and c != "Ticker"]
y_col = [target_col]

# Indices mapping (0-based)
idx_pred = [0]
idx_val = [1, 2]
idx_train = list(range(3, len(df_model)))

# Extract numpy arrays
X = df_model[X_cols].values
y = df_model[y_col].values

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_raw = X[idx_train]
y_train_raw = y[idx_train]

scaler_X.fit(X_train_raw)
scaler_y.fit(y_train_raw)

X_scaled = scaler_X.transform(X)

# Helper to create tensors
def create_tensors(indices, is_target=False):
    if is_target:
        subset = y[indices]
        subset_scaled = scaler_y.transform(subset)
        return torch.tensor(subset_scaled, dtype=torch.float32)
    else:
        subset = X_scaled[indices]
        # Reshape for LSTM: (batch_size, seq_len, input_size)
        return torch.tensor(subset, dtype=torch.float32).unsqueeze(1)

X_train_t = create_tensors(idx_train)
y_train_t = create_tensors(idx_train, is_target=True)
X_val_t = create_tensors(idx_val)
y_val_t = create_tensors(idx_val, is_target=True)
X_pred_t = create_tensors(idx_pred)

# 2. Define LSTM Model
class StockLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=32, num_layers=2, output_size=1, dropout_prob=0.3):
        super(StockLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc_1 = nn.Linear(hidden_size, 16)
        self.relu = nn.ReLU()
        self.fc_2 = nn.Linear(16, output_size)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :] # Last time step
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc_2(out)
        return out

# Initialize Model
input_features = X_train_t.shape[2]
torch.manual_seed(42) # For reproducibility
model = StockLSTM(input_features, hidden_size=32, num_layers=2)

# 3. Train
criterion = nn.MSELoss()
# Lower learning rate for longer training, L2 regularization
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
# Scheduler to decay LR when validation loss plateaus
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1000, factor=0.5)

epochs = 25000
print(f"Starting training for {epochs} epochs...")

best_val_loss = float('inf')
best_model_wts = copy.deepcopy(model.state_dict())

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()
    
    # Validation step
    model.eval()
    with torch.no_grad():
        val_out = model(X_val_t)
        val_loss = criterion(val_out, y_val_t)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            
    # Update scheduler
    scheduler.step(val_loss)
    
    if (epoch+1) % 1000 == 0:
        # Calculate R2 for display
        train_pred = outputs.detach().numpy()
        train_true = y_train_t.numpy()
        train_r2 = r2_score(train_true, train_pred)
        
        val_pred = val_out.numpy()
        val_true = y_val_t.numpy()
        if len(val_true) > 1:
            val_r2 = r2_score(val_true, val_pred)
        else:
            val_r2 = np.nan

        print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.6f} | Val Loss: {val_loss.item():.6f} | Train R2: {train_r2:.4f} | Val R2: {val_r2:.4f}")

print("\nTraining complete.")
print(f"Best Validation Loss: {best_val_loss:.6f}")

# Load best model weights
model.load_state_dict(best_model_wts)

# 4. Predict
model.eval()
with torch.no_grad():
    prediction_scaled = model(X_pred_t)
    prediction = scaler_y.inverse_transform(prediction_scaled.numpy())

print(f"\nPrediction for Row 1 (Index 0) using Best Model: {prediction[0][0]:.4f}")

ValueError: Found array with 0 sample(s) (shape=(0, 13)) while a minimum of 1 is required by MinMaxScaler.